## Tests to see if classifier works

In [1]:
import sys
import os
from pathlib import Path
DIR = Path(".").absolute()
sys.path.append(str(DIR))
os.chdir(str(DIR.parent)) # make notebook assume its in parent dir

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from model import Net
from sklearn.datasets import make_classification

In [3]:
from dask.distributed import Client

def _prep():
    from distributed.protocol import torch

client = Client(processes=False)
client.run(_prep)
client

/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: inproc://172.31.40.124/4400/1 Dashboard: http://172.31.40.124:42447/status,Cluster Workers: 1 Cores: 4 Memory: 16.48 GB


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
def get_model_weights(model):
    s = 0
    for param in model.parameters():
        s += torch.abs(torch.sum(param)).item()
    return s

In [6]:
batch_size = 128
n_updates = 5
device_str = "cpu" if not torch.cuda.is_available() else "cuda:0"
device = torch.device(device_str)
kwargs = {
    'batch_size': batch_size, 
    'max_epochs': 1, 
    'random_state': 42,
    'module': Net,
    'weight_decay': 1e-5,
    'loss': nn.CrossEntropyLoss,
    'optimizer': optim.Adagrad,
    'device': device_str
}


In [7]:
# Expected 4-dimensional input for 4-dimensional weight [32, 1, 3, 3], but got 2-dimensional input of size [32, 20] instead
X, y = make_classification(n_samples=batch_size * n_updates, n_features=784)
# match MNIST X
X.resize((640, 28, 28))
X = torch.from_numpy(X)
X = X.unsqueeze(1)
X = X.float()
X.to(device)
# match MNIST y
y = torch.from_numpy(y)
y.to(device)

print(X.shape)
print(y.shape)

torch.Size([640, 1, 28, 28])
torch.Size([640])


In [48]:
from adadamp import DaskClassifier

In [51]:
# call fit with whole dataset
est1 = DaskClassifier(**kwargs)
est1.fit(X, y)
print(get_model_weights(est1.module_))

2747.1799091994762


In [39]:
# run partial fit many times
est2 = DaskClassifier(**kwargs)
for k in range(n_updates):
    idx = np.arange(batch_size * k, batch_size * (k + 1)).astype(int)
    est2.partial_fit(X[idx], y[idx])
print(get_model_weights(est2.module_))

set rand
2078.9833680465817


In [31]:
assert np.allclose(get_model_weights(est1.module_), get_model_weights(est2.module_))

AssertionError: 